# Quantum Phase Estimation: Implementation tutorial

The quantum phase estimation algorithm is used to estimate the phase of a given operator U. Specifically, the problem statement is the following:

Given a unitary transformation $U$ on $n$ qubits and $|\psi\rangle$ is an eigenvector of $U$ with eigenvalue $e^{2\pi i\phi}$ where $0 \le \phi \le 1$. Consider that $U$ or $|\psi\rangle$ or $e^{2\pi i\phi}$ is not explicitly known but instead control-$U^{2^j}$ operations can be performed where $j \in \mathbb{R}: 0 \le j$. Also, assume that a single preparation of the state $|\psi\rangle$ is given. The problem is to obtain an m-bit estimator of $\phi$.

## Quantum Phase Estimation using Qibo

In this notebook, the implementation of the quantum phase estimation circuit using Qibo, an open source framework for programming quantum computers.

In this case, we need to import the following libraries. Especifically we need to import the different objects that we are going to use from Qibo and math, the python library.

In [ ]:
from qibo import gates
from qibo.models.circuit import Circuit
from qibo.models import QFT
import math

### Defining the input eigenvector $|\psi \rangle$

Before starting with the implementation of this circuit, we need to implement a smaller circuit. This circuit will define the input eigenvector $|\psi \rangle$ of the quantum phase estimation. In the following cell you will be able to define the circuit you want to use. A small circuit will be given and commented at the beginning of the code cell in case you don't have anything in mind.

In [ ]:
# circuit = Circuit(1)
# circuit.add(gates.X(0))

#number of qubits of your circuit
n = ...
circuit = Circuit(n)

#################
# add the needed gates to your circuit
#################

### Defining the operator U

After defining the input of our circuit. We need to define the operator we are going to use. We will estime its phase.

In the following code cell you can define the operator you want to use. You will have an option to use commented at the beginning of the cell.

You will be able to define a matrix to use as operator too.

In [ ]:
# operator = gates.T
operator = ...

# matrix = np.ndarray([[1,0],[0,math.e**(1j*math.pi/4)]])
matrix = ...

### QPE circuit

Now we have set the input eigenvector $|\psi \rangle$ of our QPE circuit, we can start building the actual circuit. The first step will be to declare the circuit. This circuit will have $mqubits$ qubits (the number of qubits that will estimate the phase we are looking for) plus the number of circuit of our input circuit defined previously.

In [ ]:
qpe = Circuit(mqubits+circuit.nqubits)

Once the circuit is declared, we have to attach the input circuit to our QPE circuit. This circuit will be applied in the last qubits. Using Qibo, we will use the $add$ method to add the input circuit in the qubits from the $mqubits$ qubit to the last qubit. We do so in the following cell.

In [ ]:
#The circuit that gives the state that will be the input of the QPE is attached to the whole QPE circuit
qpe.add(circuit.on_qubits(*range(mqubits,mqubits+circuit.nqubits)))

Now we have our circuit declared and our input circuit added, it is time to build the QPE circuit. Now our circuit will represent the following quantum state:
$$ |\psi_{0}\rangle = |0\rangle^{\otimes m}|\psi \rangle $$

The first step is to become this state to:
$$|\psi_{1}\rangle = H^{\otimes m}|0\rangle^{\otimes m}|\psi\rangle = \frac{1}{2^{\frac{m}{2}}} (|0\rangle+|1\rangle)^{\otimes m} |\psi\rangle$$

This is transform the $mqubits$ from $|0\rangle$ to its superposition applying the Hadamard gate. We will apply them in the following code cell.

In [ ]:
#The H gates are added to the firsts mqubits
for q in range(0,mqubits):
    if q < mqubits: qpe.add(gates.H(q))  

Our next step is to transform our current quantum state to the following:

$$|\psi_{1}\rangle \mapsto |\psi_{2}\rangle  =\frac {1}{2^{\frac{m}{2}}}\left(|0\rangle+{e^{2\pi i \phi 2^{m-1}}}|1\rangle \right) \otimes \cdots \otimes \left(|0\rangle+{e^{2\pi i \phi 2^{0}}}\vert1\rangle \right) \otimes |\psi\rangle = \frac{1}{2^{{\frac{m}{2}}}}\sum _{k=0}^{2^{m}-1}e^{2\pi i \phi k}|k\rangle \otimes \vert\psi\rangle
$$

In order to achieve it, we have to define and apply controlled-$U^{2^j}$. Applying the controlled-$U^{2^j}$ gate is the same as applying the controlled-$U$ gate $2^j$ times. What we achieve applying these gates, is to kick the phase of the controlled qubit to the control qubit. This phenomena is called phase kickback and will be key to achieve the quantum state we are trying to get.

In the following code cells we apply the controlled-$U$ gates. THe controlled qubits of the controlled-$U$ gate will be the qubits that represent the eigenvector $|\psi \rangle$.

Use the first cell if you defined an operator and the second cell if you defined a matrix.

In [ ]:
###################### OPERATOR OPTION ######################

#The U gates are set
reps = 2**(mqubits-1)
for q in range(mqubits):
    for i in range(reps):
        qpe.add(operator(*range(mqubits, circuit.nqubits+mqubits)).controlled_by(q))
    reps //= 2

###################### OPERATOR OPTION ######################

In [ ]:
###################### MATRIX OPTION ######################


#The U gates are set
reps = 2**(mqubits-1)
for q in range(mqubits):
  for i in range(reps):
      qpe.add(gates.Unitary(matrix,*range(mqubits, circuit.nqubits+mqubits)).controlled_by(q))
  reps //= 2

###################### MATRIX OPTION ######################

Now, if we take a look to our current state in the circuit which is:
$$|\psi_{2}\rangle   = \frac{1}{2^{{\frac{m}{2}}}}\sum _{k=0}^{2^{m}-1}e^{2\pi i \phi k}|k\rangle \otimes \vert\psi\rangle
$$

We can notice that is very similar as the output expression of the Quantum Fourier Transform. What we are going to do now is to apply the invers QFT in order to get the phase we are looking for. Thus, we will gate the following state:
$$    |\psi_3\rangle = |2^m \phi \rangle \otimes |\psi\rangle $$

In the following code cell we will apply the inverse QFT to our circuit. We will use the QFT model from Qibo.

In [ ]:
#Apply the IQFT
qft = QFT(mqubits)
iqft = qft.invert()
qpe.add(iqft.on_qubits(*range(0, mqubits)))

Now we have the implementation of the Quantum Phase Estimation. The following cell will give you a markdown that draws our final circuit.

In [ ]:
print(qpe.draw())